**目录**：
- [1.前言](#前言)
- [2.基本分类和布局](#基本分类和布局)
- [3.标量和向量](#标量和向量)
- [4.标量和矩阵](#标量和矩阵)
- [5.向量和向量](#向量和向量)
- [6.向量和矩阵](#向量和矩阵)

# 前言

在 SymPy 中，除了对单个表达式的微积分，还支持对矩阵的微积分操作

其中标量、向量、矩阵之间的求导，与神经网络中的反向传播等过程息息相关

由本文档来介绍相关的内容

In [1]:
from sympy import *
x, y, z = symbols('x y z')

# 基本分类和布局

## 类型分类

我们的讨论分为标量、向量、矩阵三种类型，并以 y 作为求导的函数，x 作为求导的自变量

$x$：标量，单个变量

$y$：标量，单个函数，例如 $y=x^2+1$

$\textbf x$：n * 1 维列向量，例如 $\textbf x=\begin{bmatrix}x_1\\x_2\\x_3\end{bmatrix}$

$\textbf y$：m * 1 维列向量，例如 $\textbf y=\begin{bmatrix}y_1=x_1+x_2\\y_2=x_2+x_3\\y_3=x_1-x_3\end{bmatrix}$

$\textbf X$：a * b 维矩阵，例如 $\textbf X=\begin{bmatrix}x & y\\z & t\end{bmatrix}$

$\textbf Y$：p * q 维矩阵，例如 $\textbf Y=\begin{bmatrix}x & x^2\\x+y & x-z\\yt & (x-1)(t+1)\end{bmatrix}$

## 求导布局

最基本的求导布局分为分子布局（numerator layout）和分母布局（denominator layout）

对于分子布局，列向量对标量的导数为列向量，标量对列向量的导数为行向量，例如：

$\textbf y=\begin{bmatrix}x\\x^2\end{bmatrix}$，则 $\dfrac{\partial\textbf y}{\partial x}=\begin{bmatrix}1\\2x\end{bmatrix}$

$\textbf x=\begin{bmatrix}x_1\\x_2\end{bmatrix},y=2x_1+3x_2$，则 $\dfrac{\partial y}{\partial\textbf x}=\begin{bmatrix}2&3\end{bmatrix}$

对于分母布局则相反，他们之间相差一个转置：

$\dfrac{\partial\textbf y}{\partial x}=\begin{bmatrix}1&2x\end{bmatrix}$

$\dfrac{\partial y}{\partial\textbf x}=\begin{bmatrix}2\\3\end{bmatrix}$

矩阵运算的布局也类似，这里不将全部情况展开叙述，参见：

https://en.wikipedia.org/wiki/Matrix_calculus#Layout_conventions

https://blog.csdn.net/keeppractice/article/details/107231685

## SymPy 的混合布局

在 SymPy 中并非统一使用分子或者分母布局，而是采用一种混合布局：

- 标量和向量的导数，会以向量的形状作为标准，保持原来的行/列形式

- 标量和矩阵的导数，也会保持矩阵原来的形状，不会作转置

- 向量对向量的导数，会产生一个 Jacobi 矩阵，此时采用的是分子布局（在后面展开介绍）

- 向量对矩阵的导数、矩阵对向量的导数、矩阵对矩阵的导数，这三种情况会产生高维的数组

后三种情况的排列布局，在网上各种资料中都没有进行详细介绍，在本文后面的部分进行介绍

总结来说，标量、向量、矩阵三种类型之间的运算布局，如下表所示：

|                    | 标量 y                                                       | 列向量 $\textbf y$                                           | 矩阵 $\textbf Y$                                             |
| ------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| 标量 $x$           | $\dfrac{\partial y}{\partial x}$：标量                       | $\dfrac{\partial\textbf y}{\partial x}$：m * 1 列向量（分子） | $\dfrac{\partial\textbf Y}{\partial x}$：p * q 矩阵（分子）  |
| 列向量 $\textbf x$ | $\dfrac{\partial y}{\partial\textbf x}$：n * 1 列向量（分母） | $\dfrac{\partial\textbf y}{\partial\textbf x}$：m * n 矩阵（分子） | $\dfrac{\partial\textbf Y}{\partial\textbf x}$：p * q * n * 1 数组 |
| 矩阵 $\textbf X$   | $\dfrac{\partial y}{\partial\textbf X}$：a * b 矩阵（分母）  | $\dfrac{\partial\textbf y}{\partial\textbf X}$：a * b * m * 1 数组 | $\dfrac{\partial\textbf Y}{\partial\textbf X}$：p * q * a * b 数组 |

# 标量和向量

在 SymPy 使用 `Matrix([list])` 的形式，由列表生成一个列向量，相当于一个 (n, 1) 的矩阵

如果是 `Matrix([[list]])` 的形式则是声明行向量

In [2]:
display(Matrix([1, 2, 3]))
display(Matrix([[1, 2, 3]]))

Matrix([
[1],
[2],
[3]])

Matrix([[1, 2, 3]])

对于函数和自变量，分别声明两个向量：

$\textbf x$： n = 3 维，列向量

$\textbf y$： m = 4 维，列向量

使用符号列表形式来生成 $\textbf x$

In [3]:
X = Matrix(list(symbols('x_1:4')))
display(X)
display(shape(X))

Matrix([
[x_1],
[x_2],
[x_3]])

(3, 1)

对于函数 $\textbf y$ 的向量，每个元素都是 $\textbf x$ 的一个函数

在 SymPy 声明函数需要使用 `y = Function('y')(x1, x2, ...)` 的方式，这里使用 `*list(X)` 向 $\textbf y$ 传入 $\textbf x$ 中的符号

$\textbf y=\begin{bmatrix}y_1=f_1(\textbf x)\\\cdots\\y_4=f_4(\textbf x)\end{bmatrix}$

In [4]:
Y = Matrix([Function('y_'+str(i))(*list(X)) for i in range(1, 5)])
display(Y)
display(shape(Y))

Matrix([
[y_1(x_1, x_2, x_3)],
[y_2(x_1, x_2, x_3)],
[y_3(x_1, x_2, x_3)],
[y_4(x_1, x_2, x_3)]])

(4, 1)

也可以将 $\textbf y$ 转置为行向量形式

In [5]:
Y.T

Matrix([[y_1(x_1, x_2, x_3), y_2(x_1, x_2, x_3), y_3(x_1, x_2, x_3), y_4(x_1, x_2, x_3)]])

## 标量对向量求导（分母布局）

$\textbf x$： n = 3 维，列向量

$\dfrac{\partial y_1}{\partial\textbf x} = 
  \begin{bmatrix} \dfrac{\partial y_1}{\partial x_1} \\ \cdots \\ \dfrac{\partial y_1}{\partial x_n} \end{bmatrix}$

得到 n = 3 维列向量

In [6]:
diff(Y[0], X)

Matrix([
[Derivative(y_1(x_1, x_2, x_3), x_1)],
[Derivative(y_1(x_1, x_2, x_3), x_2)],
[Derivative(y_1(x_1, x_2, x_3), x_3)]])

SymPy 默认采用的是分母布局，即标量对列向量求导，返回列向量

如果要使用分母布局，需要对结果或者对 $\textbf x$ 进行转置

In [7]:
diff(Y[0], X.T)

Matrix([[Derivative(y_1(x_1, x_2, x_3), x_1), Derivative(y_1(x_1, x_2, x_3), x_2), Derivative(y_1(x_1, x_2, x_3), x_3)]])

## 向量对标量求导（分子布局）

$\textbf y$： m = 4 维，列向量

$\dfrac{\partial\textbf y}{\partial x_1} = 
  \begin{bmatrix} \dfrac{\partial y_1}{\partial x_1} \\ \cdots \\ \dfrac{\partial y_m}{\partial x_1} \end{bmatrix}$

得到 m = 4 维列向量

In [8]:
display(diff(Y, X[0]))

Matrix([
[Derivative(y_1(x_1, x_2, x_3), x_1)],
[Derivative(y_2(x_1, x_2, x_3), x_1)],
[Derivative(y_3(x_1, x_2, x_3), x_1)],
[Derivative(y_4(x_1, x_2, x_3), x_1)]])

# 标量和矩阵

$\textbf X$： m \* n = 2 \* 3 维

$\textbf Y$： p \* q = 2 \* 3 维

## 标量对矩阵求导（分母布局）

$\textbf X$： m \* n = 2 \* 3 维

$\dfrac{\partial y}{\partial\textbf X} = 
  \begin{bmatrix} 
      \dfrac{\partial y}{\partial x_{11}} & \cdots & \dfrac{\partial y}{\partial x_{1n}} \\ 
      \cdots \\ 
      \dfrac{\partial y}{\partial x_{m1}} & \cdots & \dfrac{\partial y}{\partial x_{mn}} 
  \end{bmatrix}$

得到 m \* n = 2 \* 3 维

首先定义矩阵 $\textbf X$

In [9]:
lX = symbols('x_11, x_12, x_13, x_21, x_22, x_23')
X = Matrix([[lX[0], lX[1], lX[2]], [lX[3], lX[4], lX[5]]])
X

Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23]])

定义 $y = f(\textbf X)$

In [10]:
y = Function('y')(*lX)

计算导数的结果采用分母布局，和原矩阵的形状保持一致，相当于 $y$ 对 $\textbf X$ 中的每个元素求导

In [11]:
diff(y, X)

Matrix([
[Derivative(y(x_11, x_12, x_13, x_21, x_22, x_23), x_11), Derivative(y(x_11, x_12, x_13, x_21, x_22, x_23), x_12), Derivative(y(x_11, x_12, x_13, x_21, x_22, x_23), x_13)],
[Derivative(y(x_11, x_12, x_13, x_21, x_22, x_23), x_21), Derivative(y(x_11, x_12, x_13, x_21, x_22, x_23), x_22), Derivative(y(x_11, x_12, x_13, x_21, x_22, x_23), x_23)]])

## 矩阵对标量求导（分子布局）

$\textbf Y$： p \* q = 2 \* 3 维

$\dfrac{\partial\textbf Y}{\partial x} = 
  \begin{bmatrix} 
      \dfrac{\partial y_{11}}{\partial x} & \cdots & \dfrac{\partial y_{1q}}{\partial x} \\ 
      \cdots \\ 
      \dfrac{\partial y_{p1}}{\partial x} & \cdots & \dfrac{\partial y_{pq}}{\partial x} 
  \end{bmatrix}$

得到 p \* q = 2 \* 3 维

首先定义函数矩阵 $\textbf Y$

In [12]:
Y = Matrix([[x, x**2, x**3], [2*x, 2*x**2, 2*x**3]])
Y

Matrix([
[  x,   x**2,   x**3],
[2*x, 2*x**2, 2*x**3]])

矩阵对标量求导，可以使用矩阵特有的 `Y.diff(x)` 或者通用的 `diff(Y, x)`，二者是等价的

返回的矩阵大小也与原来保持一致，相当于 $\textbf Y$ 的每个函数对 $x$ 求导

In [13]:
diff(Y, x) # Y.diff(x)

Matrix([
[1, 2*x, 3*x**2],
[2, 4*x, 6*x**2]])

# 向量和向量

## 向量对向量求导（分子布局）

向量对向量的求导，相当于两个向量的元素一一求导，结果会从两个列向量生成一个 Jacobi 矩阵

$\textbf y$： n = 3 维，列向量

$\textbf x$： m = 4 维，列向量

$\dfrac{\partial\textbf y}{\partial\textbf x} = 
  \begin{bmatrix} 
      \dfrac{\partial y_1}{\partial x_1} & \cdots & \dfrac{\partial y_1}{\partial x_n} \\ 
      \cdots \\ 
      \dfrac{\partial y_m}{\partial x_1} & \cdots & \dfrac{\partial y_m}{\partial x_n} 
  \end{bmatrix}$
  
得到 m \* n = 4 \* 3 维矩阵

首先定义 $\textbf x$ 和 $\textbf y$

In [14]:
X = Matrix(list(symbols('x_1:4')))
Y = Matrix([Function('y_'+str(i))(*list(X)) for i in range(1, 5)])

display(X)
display(Y)

Matrix([
[x_1],
[x_2],
[x_3]])

Matrix([
[y_1(x_1, x_2, x_3)],
[y_2(x_1, x_2, x_3)],
[y_3(x_1, x_2, x_3)],
[y_4(x_1, x_2, x_3)]])

### 使用 jacobian

SymPy 内置了 `jacobian` 函数，可以用于计算指定向量的 jacobi 矩阵

两个操作对象必须是行或列向量

向量对自己本身求导，会产生单位阵

In [15]:
X.jacobian(X)

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

向量对向量求导，会按分子布局排列

In [16]:
Y.jacobian(X)

Matrix([
[Derivative(y_1(x_1, x_2, x_3), x_1), Derivative(y_1(x_1, x_2, x_3), x_2), Derivative(y_1(x_1, x_2, x_3), x_3)],
[Derivative(y_2(x_1, x_2, x_3), x_1), Derivative(y_2(x_1, x_2, x_3), x_2), Derivative(y_2(x_1, x_2, x_3), x_3)],
[Derivative(y_3(x_1, x_2, x_3), x_1), Derivative(y_3(x_1, x_2, x_3), x_2), Derivative(y_3(x_1, x_2, x_3), x_3)],
[Derivative(y_4(x_1, x_2, x_3), x_1), Derivative(y_4(x_1, x_2, x_3), x_2), Derivative(y_4(x_1, x_2, x_3), x_3)]])

### 使用 diff


如果使用通用的 diff 进行求导，则会产生一个多维数组

In [17]:
D = diff(X, X)
display(D)
display(D.shape)

[[[[1], [0], [0]]], [[[0], [1], [0]]], [[[0], [0], [1]]]]

(3, 1, 3, 1)

多维数组的类型为 N-dim Array，需要使用 `reshape` 整理形状并使用 `tomatrix` 转换为矩阵

参见：

https://docs.sympy.org/latest/modules/tensor/array.html?highlight=tomatrix

In [18]:
D.reshape(3, 3).tomatrix()

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

$\textbf y$ 对 $\textbf x$ 求导也是同理

In [19]:
D = diff(Y, X)
display(D.shape)
display(D)

(3, 1, 4, 1)

[[[[Derivative(y_1(x_1, x_2, x_3), x_1)], [Derivative(y_2(x_1, x_2, x_3), x_1)], [Derivative(y_3(x_1, x_2, x_3), x_1)], [Derivative(y_4(x_1, x_2, x_3), x_1)]]], [[[Derivative(y_1(x_1, x_2, x_3), x_2)], [Derivative(y_2(x_1, x_2, x_3), x_2)], [Derivative(y_3(x_1, x_2, x_3), x_2)], [Derivative(y_4(x_1, x_2, x_3), x_2)]]], [[[Derivative(y_1(x_1, x_2, x_3), x_3)], [Derivative(y_2(x_1, x_2, x_3), x_3)], [Derivative(y_3(x_1, x_2, x_3), x_3)], [Derivative(y_4(x_1, x_2, x_3), x_3)]]]]

In [20]:
D.reshape(4, 3).tomatrix()

Matrix([
[Derivative(y_1(x_1, x_2, x_3), x_1), Derivative(y_2(x_1, x_2, x_3), x_1), Derivative(y_3(x_1, x_2, x_3), x_1)],
[Derivative(y_4(x_1, x_2, x_3), x_1), Derivative(y_1(x_1, x_2, x_3), x_2), Derivative(y_2(x_1, x_2, x_3), x_2)],
[Derivative(y_3(x_1, x_2, x_3), x_2), Derivative(y_4(x_1, x_2, x_3), x_2), Derivative(y_1(x_1, x_2, x_3), x_3)],
[Derivative(y_2(x_1, x_2, x_3), x_3), Derivative(y_3(x_1, x_2, x_3), x_3), Derivative(y_4(x_1, x_2, x_3), x_3)]])

# 向量和矩阵

网络上的大部分资料都没有向量对矩阵、矩阵对向量、矩阵对矩阵这三种类型的介绍

按照 wikipedia ，这三种计算还没有形成共识

但是在神经网络中，例如我们想要计算结果（向量）对权重（矩阵）的导数，则会涉及到这种计算

本质上它们都是向量/矩阵之间，每个元素一一对应地进行计算，但是在结果的布局上没有统一的说法

In [21]:
x, y, z, t = symbols('x y z t')

## 向量对矩阵求导（混合布局）

向量对矩阵的求导，可以参考标量对矩阵的求导，将向量与矩阵的每个元素（标量）求导，然后再对向量的每个元素（标量）进行求导

$\textbf y$： m = 2 维，列向量

$\textbf X$： p \* q = 2 \* 3 维

按照 SymPy 的默认混合布局：

首先会对分母进行展开

$\dfrac{\partial\textbf y}{\partial\textbf X}$ 为 p * q = 2 * 3 维矩阵 （标量-矩阵，分母布局）

然后再对分子进行展开

$\dfrac{\partial\textbf y}{\partial x_{ij}}$ 为 m = 2 维列向量 （向量-标量，分子布局）

$\dfrac{\partial\textbf y}{\partial\textbf X}
= 
\begin{bmatrix} 
  \dfrac{\partial\textbf y}{\partial x_{11}} & \cdots & \dfrac{\partial\textbf y}{\partial x_{1q}} \\ 
  \cdots \\ 
  \dfrac{\partial\textbf y}{\partial x_{p1}} & \cdots & \dfrac{\partial\textbf y}{\partial x_{pq}} 
\end{bmatrix}
= 
\begin{bmatrix} 
  \begin{bmatrix}\dfrac{\partial y_1}{\partial x_{11}}\\\cdots\\\dfrac{\partial y_m}{\partial x_{11}}\end{bmatrix} 
    & \cdots 
    & \begin{bmatrix}\dfrac{\partial y_1}{\partial x_{1q}}\\\cdots\\\dfrac{\partial y_m}{\partial x_{1q}}\end{bmatrix} 
  \\ \cdots \\ 
  \begin{bmatrix}\dfrac{\partial y_1}{\partial x_{p1}}\\\cdots\\\dfrac{\partial y_m}{\partial x_{p1}}\end{bmatrix} 
    & \cdots 
    & \begin{bmatrix}\dfrac{\partial y_1}{\partial x_{pq}}\\\cdots\\\dfrac{\partial y_m}{\partial x_{pq}}\end{bmatrix} 
\end{bmatrix}$

我们定义一个 vy 和 mX，使用 `diff` 进行计算

In [22]:
vy = Matrix([x * y * z * t, 2*x + y**2 + log(z) + sin(t)])
mX = Matrix([[x, y, z], [y, z, t]])
display(vy)
display(mX)

Matrix([
[                     t*x*y*z],
[2*x + y**2 + log(z) + sin(t)]])

Matrix([
[x, y, z],
[y, z, t]])

结果会返回一个 p \* q \* m = 2 \* 3 \* 2 的多维数组

In [23]:
D = diff(vy, mX)
display(D)
display(D.shape)

[[[[t*y*z], [2]], [[t*x*z], [2*y]], [[t*x*y], [1/z]]], [[[t*x*z], [2*y]], [[t*x*y], [1/z]], [[x*y*z], [cos(t)]]]]

(2, 3, 2, 1)

将结果转为矩阵

In [24]:
M = zeros(4, 3)
for i in range(2):
    for j in range(3):
        for k in range(2):
            M[(i*2+k)*3+j] = D[i][j][k][0]           
M

Matrix([
[t*y*z, t*x*z,  t*x*y],
[    2,   2*y,    1/z],
[t*x*z, t*x*y,  x*y*z],
[  2*y,   1/z, cos(t)]])

### 转为分子布局

如果统一按照分子布局，则需要使用 $\textbf X$ 的转置来计算：

$\dfrac{\partial\textbf y}{\partial\textbf X^T}$ 为 q * p = 3 * 2 维矩阵（标量-矩阵，分子布局）

$\dfrac{\partial\textbf y}{\partial x_{ji}}$ 为 m = 2 维列向量（向量-标量，分子布局）

In [25]:
D = diff(vy, mX.T)
display(D)
display(D.shape)

[[[[t*y*z], [2]], [[t*x*z], [2*y]]], [[[t*x*z], [2*y]], [[t*x*y], [1/z]]], [[[t*x*y], [1/z]], [[x*y*z], [cos(t)]]]]

(3, 2, 2, 1)

将结果转为矩阵

In [26]:
M = zeros(6, 2)
for i in range(3):
    for j in range(2):
        for k in range(2):
            M[(i*2+k)*2+j] = D[i][j][k][0]           
M

Matrix([
[t*y*z,  t*x*z],
[    2,    2*y],
[t*x*z,  t*x*y],
[  2*y,    1/z],
[t*x*y,  x*y*z],
[  1/z, cos(t)]])

### 转为分母布局

如果统一按照分母布局，则需要使用 $\textbf y$ 的转置来计算：

$\dfrac{\partial\textbf y^T}{\partial\textbf X}$ 为 p * q = 2 * 3 维矩阵（标量-矩阵，分母布局）

$\dfrac{\partial\textbf y^T}{\partial x_{ij}}$ 为 m = 2 维行向量（向量-标量，分母布局）

In [27]:
diff(vy.T, mX)

[[[[t*y*z, 2]], [[t*x*z, 2*y]], [[t*x*y, 1/z]]], [[[t*x*z, 2*y]], [[t*x*y, 1/z]], [[x*y*z, cos(t)]]]]

## 矩阵对向量求导

$\textbf x$： n = 2 维，列向量

$\textbf Y$： p \* q = 2 \* 3 维

按照 SymPy 的默认混合布局：

首先会对分母进行展开

$\dfrac{\partial\textbf Y}{\partial\textbf x}$ 为 n = 2 维列向量 （标量-向量，分母布局）

然后再对分子进行展开

$\dfrac{\partial\textbf Y}{\partial x_i}$ 为 p * q = 2 * 3 维矩阵 （矩阵-标量，分子布局）

$\dfrac{\partial\textbf Y}{\partial\textbf x}
= 
\begin{bmatrix} 
  \dfrac{\partial\textbf Y}{\partial x_1} \\ \cdots \\ \dfrac{\partial\textbf Y}{\partial x_n} 
\end{bmatrix}
= 
\begin{bmatrix} 
  \begin{bmatrix} 
    \dfrac{\partial y_{11}}{\partial x_1} & \cdots & \dfrac{\partial y_{1q}}{\partial x_1} 
    \\ \cdots \\ 
    \dfrac{\partial y_{p1}}{\partial x_1} & \cdots & \dfrac{\partial y_{pq}}{\partial x_1} 
  \end{bmatrix} 
  \\ 
  \cdots 
  \\ 
  \begin{bmatrix} 
    \dfrac{\partial y_{11}}{\partial x_n} & \cdots & \dfrac{\partial y_{1q}}{\partial x_n} 
    \\ \cdots \\ 
    \dfrac{\partial y_{p1}}{\partial x_n} & \cdots & \dfrac{\partial y_{pq}}{\partial x_n} 
  \end{bmatrix}
\end{bmatrix}
$

In [28]:
X = Matrix([x, y])
Y = Matrix([[1 * x * y, 1 * x**2 * y, 1 * x * y**2], 
            [2 * x + y, 2 * x**2 + y, 2 * x + y**2]])
display(X)
display(Y)

Matrix([
[x],
[y]])

Matrix([
[    x*y,     x**2*y,     x*y**2],
[2*x + y, 2*x**2 + y, 2*x + y**2]])

In [29]:
D = diff(Y, X)
D

[[[[y, 2*x*y, y**2], [2, 4*x, 2]]], [[[x, x**2, 2*x*y], [1, 1, 2*y]]]]

In [30]:
D.shape

(2, 1, 2, 3)

采用混合布局，直接 reshape 即可

In [31]:
D.reshape(4, 3).tomatrix()

Matrix([
[y, 2*x*y,  y**2],
[2,   4*x,     2],
[x,  x**2, 2*x*y],
[1,     1,   2*y]])

### 转为分子布局

如果统一按照分子布局，则需要使用 $\textbf x$ 的转置来计算：

$\dfrac{\partial\textbf Y}{\partial\textbf x^T}$ 为 n = 2 维行向量 （标量-向量，分子布局）

$\dfrac{\partial\textbf Y}{\partial x_i}$ 为 p * q = 2 * 3 维矩阵 （矩阵-标量，分子布局）

In [32]:
diff(Y, X.T)

[[[[y, 2*x*y, y**2], [2, 4*x, 2]], [[x, x**2, 2*x*y], [1, 1, 2*y]]]]

### 转为分母布局

如果统一按照分母布局，则需要使用 $\textbf Y$ 的转置来计算：

$\dfrac{\partial\textbf Y^T}{\partial\textbf x}$ 为 n = 2 维列向量 （标量-向量，分母布局）

$\dfrac{\partial\textbf Y^T}{\partial x_i}$ 为 q * p = 3 * 2 维矩阵 （矩阵-标量，分母布局）

In [33]:
diff(Y.T, X)

[[[[y, 2], [2*x*y, 4*x], [y**2, 2]]], [[[x, 1], [x**2, 1], [2*x*y, 2*y]]]]

## 矩阵对矩阵求导

$\textbf X$： m \* n = 2 \* 3 维

$\textbf Y$： p \* q = 2 \* 3 维

按照 SymPy 的默认混合布局：

首先会对分母进行展开

$\dfrac{\partial\textbf Y}{\partial\textbf X}$ 为 m * n = 2 * 3 维矩阵 （标量-矩阵，分母布局）

然后再对分子进行展开

$\dfrac{\partial\textbf Y}{\partial x_{ij}}$ 为 p * q = 2 * 3 维矩阵 （矩阵-标量，分子布局）

$\dfrac{\partial\textbf Y}{\partial\textbf X}
= 
\begin{bmatrix} 
  \dfrac{\partial\textbf Y}{\partial x_{11}} & \cdots & \dfrac{\partial\textbf Y}{\partial x_{1n}} 
  \\ 
  \cdots 
  \\ 
  \dfrac{\partial\textbf Y}{\partial x_{m1}} & \cdots & \dfrac{\partial\textbf Y}{\partial\textbf x_{mn}} 
\end{bmatrix}
= 
\begin{bmatrix} 
  \begin{bmatrix}
    \dfrac{\partial y_{11}}{\partial x_{11}}
      & \cdots &
    \dfrac{\partial y_{11}}{\partial x_{1n}}
    \\
    \cdots
    \\
    \dfrac{\partial y_{11}}{\partial x_{m1}}
      & \cdots &
    \dfrac{\partial y_{11}}{\partial x_{mn}}
  \end{bmatrix} 
  & \cdots 
  & \begin{bmatrix}
    \dfrac{\partial y_{1q}}{\partial x_{11}}
      & \cdots &
    \dfrac{\partial y_{1q}}{\partial x_{1n}}
    \\
    \cdots
    \\
    \dfrac{\partial y_{1q}}{\partial x_{m1}}
      & \cdots &
    \dfrac{\partial y_{1q}}{\partial x_{mn}}
  \end{bmatrix} 
  \\ 
  \cdots 
  \\ 
  \begin{bmatrix}
    \dfrac{\partial y_{p1}}{\partial x_{11}}
      & \cdots &
    \dfrac{\partial y_{p1}}{\partial x_{1n}}
    \\
    \cdots
    \\
    \dfrac{\partial y_{p1}}{\partial x_{m1}}
      & \cdots &
    \dfrac{\partial y_{p1}}{\partial x_{mn}}
  \end{bmatrix} 
  & \cdots 
  & \begin{bmatrix}
    \dfrac{\partial y_{pq}}{\partial x_{11}}
      & \cdots &
    \dfrac{\partial y_{pq}}{\partial x_{1n}}
    \\
    \cdots
    \\
    \dfrac{\partial y_{pq}}{\partial x_{m1}}
      & \cdots &
    \dfrac{\partial y_{pq}}{\partial x_{mn}}
  \end{bmatrix} 
\end{bmatrix}
$

In [34]:
x, y, z, t = symbols('x y z t')
X = Matrix([[x, y, z], [y, z, x]])
Y = Matrix([[2*x+y+z, x+2*y+z, x+y+2*z], 
            [x**2*y*z, x*y**2*z, x*y*z**2]])
display(X)
display(Y)

Matrix([
[x, y, z],
[y, z, x]])

Matrix([
[2*x + y + z, x + 2*y + z, x + y + 2*z],
[   x**2*y*z,    x*y**2*z,    x*y*z**2]])

In [35]:
D = diff(Y, X)
D

[[[[2, 1, 1], [2*x*y*z, y**2*z, y*z**2]], [[1, 2, 1], [x**2*z, 2*x*y*z, x*z**2]], [[1, 1, 2], [x**2*y, x*y**2, 2*x*y*z]]], [[[1, 2, 1], [x**2*z, 2*x*y*z, x*z**2]], [[1, 1, 2], [x**2*y, x*y**2, 2*x*y*z]], [[2, 1, 1], [2*x*y*z, y**2*z, y*z**2]]]]

In [36]:
D.shape

(2, 3, 2, 3)

In [37]:
M = zeros(4, 9)
for i in range(2):
    for j in range(3):
        for k in range(2):
            for l in range(3):
                M[i*18+ j*3+ k*9 + l] = D[i][j][k][l]           
M

Matrix([
[      2,       1,      1,      1,       2,       1,       1,      1,       2],
[2*x*y*z,  y**2*z, y*z**2, x**2*z, 2*x*y*z,  x*z**2,  x**2*y, x*y**2, 2*x*y*z],
[      1,       2,      1,      1,       1,       2,       2,      1,       1],
[ x**2*z, 2*x*y*z, x*z**2, x**2*y,  x*y**2, 2*x*y*z, 2*x*y*z, y**2*z,  y*z**2]])

### 转为分子布局

如果统一按照分子布局，则需要使用 $\textbf X$ 的转置来计算：

$\dfrac{\partial\textbf Y}{\partial\textbf X^T}$ 为 n * m = 3 * 2 维矩阵 （标量-矩阵，分子布局）

$\dfrac{\partial\textbf Y}{\partial x_{ji}}$ 为 p * q = 2 * 3 维矩阵 （矩阵-标量，分子布局）

In [38]:
diff(Y, X.T)

[[[[2, 1, 1], [2*x*y*z, y**2*z, y*z**2]], [[1, 2, 1], [x**2*z, 2*x*y*z, x*z**2]]], [[[1, 2, 1], [x**2*z, 2*x*y*z, x*z**2]], [[1, 1, 2], [x**2*y, x*y**2, 2*x*y*z]]], [[[1, 1, 2], [x**2*y, x*y**2, 2*x*y*z]], [[2, 1, 1], [2*x*y*z, y**2*z, y*z**2]]]]

### 转为分母布局

如果统一按照分母布局，则需要使用 $\textbf Y$ 的转置来计算：

$\dfrac{\partial\textbf Y^T}{\partial\textbf X}$ 为 m * n = 2 * 3 维矩阵 （标量-矩阵，分母布局）

$\dfrac{\partial\textbf Y^T}{\partial x_{ij}}$ 为 q * p = 3 * 2 维矩阵 （矩阵-标量，分母布局）

In [39]:
diff(Y.T, X)

[[[[2, 2*x*y*z], [1, y**2*z], [1, y*z**2]], [[1, x**2*z], [2, 2*x*y*z], [1, x*z**2]], [[1, x**2*y], [1, x*y**2], [2, 2*x*y*z]]], [[[1, x**2*z], [2, 2*x*y*z], [1, x*z**2]], [[1, x**2*y], [1, x*y**2], [2, 2*x*y*z]], [[2, 2*x*y*z], [1, y**2*z], [1, y*z**2]]]]